# Mais detalhes sobre funções

## Escopo

Escopo é o nome dado à parte do código onde um identificador é válido.

Dois tipos de escopo importantes em Python são os chamados **escopo global** e **escopo local**.

O *escopo global* se aplica a todos os identificadores que são definidos em um módulo. Veremos módulos mais adiante. Por enquanto, basta saber que todos os identificadores criados diretamente no interpretador fazem parte de um módulo.

O *escopo local* se aplica a identificadores criados dentro de uma função.

Identificadores no escopo global podem ser referenciados em qualquer lugar no módulo, enquanto identificadores locais somente podem ser referenciados na função onde foram criados.

Assim, no código abaixo, `variavel_global` e `f` (o nome da função) são identificadores globais, enquanto `variavel_local`, `par1` e `par2` (os parâmetros) são identificadores locais da função `f`.

In [1]:
variavel_global = 10
def f(par1, par2):
    variavel_local = 12
    return par1 + par2 - variavel_global + variavel_local

Os identificadores globais são acessíveis no escopo do módulo (direto no interpretador, neste caso), mas os locais não.

In [2]:
variavel_global

10

In [3]:
f

<function __main__.f>

In [4]:
variavel_local

NameError: name 'variavel_local' is not defined

In [5]:
par1

NameError: name 'par1' is not defined

Cada função define um novo escopo local. Assim, quando uma função é definida dentro da outra, os identificadores locais da função externa são acessíveis na função externa (mas não o contrário).

In [6]:
def g(x):
    def h(y):
        return y ** x
    return x + h(2)


In [7]:
g(3)

11

Por outro lado, como a função define um novo escopo, se criarmos uma variável (através de uma atribuição) com o mesmo identificador de uma variável externa, ela irá esconder o valor da variável externa.

In [8]:
variavel_global = 10
def f(par1, par2):
    variavel_local = 12
    variavel_global = 8 # Esta é uma nova variável no escopo de f
    return par1 + par2 - variavel_global + variavel_local

In [9]:
f(2,3)

9

O valor de `variavel_global` não foi afetado pela execução de `f`.

In [10]:
variavel_global

10

O efeito dessas regras é que as variáveis externas podem ser acessíveis dentro de escopos internos, desde que queiramos apenas acessar os objetos a que essas variáveis se referenciam, mas sem atribuir novos objetos a essas variáveis.

Se quisermos mudar o objeto associado com a variável externa, devemos declarar isso explicitamente usando a palavra-chave `global` (para acesso a variáveis globais):

In [11]:
variavel_global = 10
def f(par1, par2):
    global variavel_global
    variavel_local = 12
    variavel_global = 8
    return par1 + par2 - variavel_global + variavel_local

Neste caso, qualquer alteraço do objeto associado a `variavel_global` feita dentro de `f` será refletida na variável global:

In [12]:
f(2,3)

9

In [13]:
variavel_global

8

Mas não se confunda: sem o uso de `global`, não podemos mudar o objeto a que a variável global se referencia, mas podemos, se o objeto for de um tipo mutável, alterar seu valor:

In [14]:
lista = []
def estraga(x):
    lista.append(x)

Ao executar essa função, adicionamos um elemento à lista referenciada pela variável global `lista`, mudando portanto seu valor.

In [15]:
estraga(2)
lista

[2]

In [16]:
estraga(3)
lista

[2, 3]

Quando uma nova variável é declarada em escopo mais interno com o mesmo nome de uma variável existente em escopo externo, ela "esconde" a variável externa.

In [17]:
x = 1
def f():
    x = 2
    def g():
        x = 3
        def h():
            print('x em h:', x)
        h()
        print('x em g:', x)
    g()
    print('x em f:', x)
print('x fora:', x)

x fora: 1


In [18]:
f()

x em h: 3
x em g: 3
x em f: 2


Se queremos nos referir em um escopo mais interno a uma variável de escopo local mais externo, devemos usar a palavra-chave `nonlocal`, que tem efeito similar, para esses casos, ao que `global` faz para variáveis globais.

In [19]:
def f():
    x = 2
    def g():
        nonlocal x
        x = 3
        print('x em g:', x)
    print('x em f (antes de g):', x)
    g()
    print('x em f (depois de g):', x)

In [20]:
f()

x em f (antes de g): 2
x em g: 3
x em f (depois de g): 3


### Closures

Uma característica interessante de funções em Python é que elas podem guardar informações sobre o ambiente onde foram definidas. Assim, se uma função `g` é definida dentro de outra função `f`, `g` pode guardar referência para os objetos de `f` de quando ela foi definida.

O termo técnico para isso é **closure**.

Como funções são objetos, elas podem ser retornadas por outra funções. Por exemplo, no código abaixo, a função `f`, quando chamada, retorna uma *closure* que lembra o objeto `x` usado em sua definição:

In [21]:
def f(x):
    def g(y):
        return x + y
    return g

In [22]:
a = f(2)

Note que agora `a` é uma função:

In [23]:
a

<function __main__.f.<locals>.g>

Podemos usá-la então passando o parâmetro que ela precisa:

In [26]:
a(1)

3

O resultado aqui foi 3, pois a função `a` soma ao valor passado para ela o valor 2, que foi passado a `f` para a criação de `a`.

In [25]:
b = f(5)
b(1)

6

Abaixo uma função que "memoriza" uma lista de números passados em execuções sucessivas:

In [2]:
def f(usados):
    def g(x):
        usados.append(x)
        print('Usados até agora:', usados)
        return len(usados)
    return g

In [3]:
a = f([])

In [4]:
a(1)

Usados até agora: [1]


1

In [30]:
a(2)

Usados até agora: [1, 2]


2

In [31]:
a(0)

Usados até agora: [1, 2, 0]


3

## Argumentos

Uma função especifica os parâmetros que serão usados em sua execução. Ao executar a função, precisamos fornecer valores para todos esses parâmetros. Como vimos, os parâmetros são variáveis, que guardarão uma referência para os objetos passados em sua chamada.

A correspondência entre os valores passados e os parâmetros pode ser feita de diversas formas em Python. A primeira e mais simples é a *posicional*, que é a mesma usada em C. Neste caso, o primeiro valor fornecido será associado ao primeiro parâmetro, o segundo valor ao segundo parâmetro, e assim por diante.

In [32]:
def f(x, y):
    print('x =', x, 'y =', y)

In [33]:
f(1,2)

x = 1 y = 2


In [34]:
f(5,55)

x = 5 y = 55


O número de valores passados deve corresponder ao número de parâmetros esperados.

In [35]:
f(1)

TypeError: f() missing 1 required positional argument: 'y'

In [36]:
f(1,2,3)

TypeError: f() takes 2 positional arguments but 3 were given

Adicionalmente, Python permite a passagem pela sintaxe `chave=valor`, onde chave é o nome de um parâmetro da função e valor é o valor que ele irá receber. Note que **nesse caso, os parâmetros não precisam ser passados na mesma ordem**.

In [37]:
f(x=1, y=2)

x = 1 y = 2


In [38]:
f(y=1, x=2)

x = 2 y = 1


Outra versatilidade ocorre pois podemos fornecer um valor *default* para um parâmetro. Neste caso, se um valor para o parâmetro não for especificado, o valor *default* será usado.

In [39]:
def f(x, y, z = 0):
    print ('x =', x, ', y =', y, ', z =', z)

In [40]:
f(1,2,3)

x = 1 , y = 2 , z = 3


In [41]:
f(z = 2, y = 4, x = 1)

x = 1 , y = 4 , z = 2


In [42]:
f(2,3)

x = 2 , y = 3 , z = 0


In [43]:
f(x=3,y=4)

x = 3 , y = 4 , z = 0


In [44]:
f(7, y=8)

x = 7 , y = 8 , z = 0


É frequente usar-se `None` como o valor default, e então sabemos que um valor não foi especificado na chamada comparando o parâmetro com `None`.

In [45]:
def f(x, y, z = None):
    if z is None:
        z = x - y
    print(x, y, z)

In [46]:
f(3,5,7)

3 5 7


In [47]:
f(3,5)

3 5 -2


Uma limitação é que na chamada os parâmetros que usam `chave=valor` devem ser colocados após todos os parâmetros posicionais desejados.

In [48]:
f(y = 1, 2)

SyntaxError: positional argument follows keyword argument (<ipython-input-48-35d04d818f68>, line 1)

In [49]:
f(2, z=1, y = 3)

2 3 1


Cada parâmetro deve receber exatamente um valor. Quer dizer, não podemos deixar sem valor nem fornecer múltiplos valores (este último caso costuma ocorrer por engano quando usamos `chave=valor` para um parâmetro que já recebeu valor posicional.

In [50]:
f(2, x = 2)

TypeError: f() got multiple values for argument 'x'

In [51]:
f(2, z = 4)

TypeError: f() missing 1 required positional argument: 'y'

Frequentemente desejamos definir funções que recebem número arbitrário de parâmetros. Um exemplo é a função do Python `min`.

In [52]:
min(1,4,7,-2,0,3)

-2

In [53]:
min(2,5)

2

Essa função só precisa ter pelo menos 2 valores para funcionar. Se ela recebe 1 valor só, espera que esse valor seja um objeto que pode ser percorrido (como por um `for`).

In [54]:
min([2,5,8,1,9,-3,4,0])

-3

In [55]:
min(1)

TypeError: 'int' object is not iterable

Podemos definir uma função similar ao `min` mas com comportamento distinto para apenas um valor usando a sintaxe de coleta de múltiplos valores:

In [56]:
def meu_min(x, *resto):
    print('x =', x, 'resto =', resto)
    minimo = x
    for y in resto:
        if y < minimo:
            minimo = y
    return minimo

Nessa função, `x` receberá o primeiro valor passado, enquanto `resto` receberá uma **tupla** com todos os outros valores:

In [57]:
meu_min(2,3,-6,10,1,7,0,5)

x = 2 resto = (3, -6, 10, 1, 7, 0, 5)


-6

In [58]:
meu_min(1)

x = 1 resto = ()


1

A sintaxe `*resto` coleta todos os valores **posicionais** passados na chamada. Se quisermos coletar valores do tipo `chave=valor` precisamos usar dois asteriscos:

In [59]:
def f(**args):
    for k in args.keys():
        print('Para', k, 'temos o valor', args[k])

In [60]:
f(altura=1.67, peso=54, idade=21)

Para idade temos o valor 21
Para altura temos o valor 1.67
Para peso temos o valor 54


Note como os valores são coletados em um *dicionário*, cujas chaves são cadeias de caracteres correspondentes às chaves na chamada da função.

É também possível coletar simultaneamente valores posicionais e `chave=valor`:

In [61]:
def f(x, *y, **z):
    print(x, y, z)

In [62]:
f(1,2,3,4,5)

1 (2, 3, 4, 5) {}


In [63]:
f(1,2,3,a=4,b=5)

1 (2, 3) {'b': 5, 'a': 4}


Os códigos acima mostram como diversos valores passados como argumentos podem ser coletados no corpo da função.

A operação inversa também é possível: Podemos pegar elementos coletados e espalhá-los para os diversos parâmetros da função.

Por exemplo, a função abaixo requer 3 parâmetros.

In [64]:
def g(x,y,z):
    return x * y - z

In [65]:
g(2,3,4)

2

Se tivermos os valores a passar na ordem certa em uma tupla, uma forma de fazer a chamada seria:

In [66]:
t = (2,3,4)
g(t[0],t[1],t[2])

2

No entanto, isso pode ser feito automaticamente pelo Python com o uso de um asterisco:

In [67]:
g(*t)

2

Para o caso `chave=valor` temos algo similar, mas os elementos devem estar num dicionário (com as chaves como cadeias de caracteres) e usamos dois asteriscos:

In [68]:
d = {'x': 2, 'y': 3, 'z': 4}

In [69]:
g(**d)

2

Uma sintaxe adicional em Python é a de parâmetros que precisam *necessariamente* ser especificados no formato `chave=valor` durante a chamada. Qualquer parâmetro normal especificado **após** um parâmetro de coleta posicional será desse tipo, como no caso do parâmetro `z` da função abaixo.

In [5]:
def h(x, *y, z = 0):
    print(x, y, z)

In [6]:
h(1,2,z=3)

1 (2,) 3


In [72]:
h(1,2,3)

1 (2, 3) 0


Quando usando valores assumidos, deve-se tomar muito cuidado com valores que sejam mutáveis (por exemplo, uma lista).

Veja o exemplo abaixo de uma função cujo parâmetro `y` é uma lista vazia se não for especificado.

In [73]:
def crazy(x, y = []):
    y.append(x)
    print(y)

In [74]:
lista = [1,2]
crazy(3,lista)

[1, 2, 3]


In [75]:
lista

[1, 2, 3]

In [76]:
crazy(4, lista)

[1, 2, 3, 4]


Até agora, tudo ocorre como esperado. Vamos ver o que acontece se não especificamos a lista:

In [77]:
crazy(4)

[4]


Por enquanto é o que esperávamos. Um 4 foi adicionado a uma lista vazia.

O que acontece se chamamos novamente sem especificar uma lista?

In [78]:
crazy(5)

[4, 5]


Note como o 5 *não foi adicionado a uma lista vazia*, mas sim à mesma lista usada para adicionar o 4.

Isto ocorre porque os objetos usados na inicialização são calculados apenas uma vez. Depois sempre que um objeto default for necessário, esse mesmo objeto será utilizado.

A regra para lidar com isso é **evitar usar objetos mutáveis em valores default**. Ao invés disso, usamos `None`, e inicializamos com um novo objeto quando necessário: 

In [79]:
def noncrazy(x, y = None):
    if y is None:
        y = []
    y.append(x)
    print(y)

In [80]:
noncrazy(6, lista)

[1, 2, 3, 4, 6]


In [81]:
noncrazy(7)

[7]


In [82]:
noncrazy(8)

[8]


### Recursão

Funções Python podem ser chamadas recursivamente. Para exemplificar, veja a implementação interessante (mas não eficiente) de quicksort abaixo:

In [83]:
def qsort(lista):
    if len(lista) <= 1: # Retorna se nada a ordenar
        return lista
    chave = lista[0] # Escolhe primeiro elemento como chave
    menores = []
    maiores = []
    # Separa maiores e menores que chave em listas
    for val in lista[1:]: # Começa no 1, pois 0 é a chave
        if val < chave:
            menores.append(val)
        else:
            maiores.append(val)
    # Ordena recursivamente menores e maiores, depois junta
    return qsort(menores) + [chave] + qsort(maiores)

In [84]:
qsort([2,4,6,9,7,5])

[2, 4, 5, 6, 7, 9]

## Funções lambda

Em muitas situações, precisamos de uma função simples, que possivelmente não será mais usada em outros lugares. Neste caso, definir uma nova função com `def` pode ser mais complicado que o necessário.

Isso é resolvido pela definição de *funções lambda*, que são funções simples que não precisam receber um nome.

A sintaxe é a seguinte:

In [85]:
f1 = lambda x, y: 2 * x - y

Esse código definiu uma função de dois parâmetros (chamados `x` e `y`) que retorna o valor da expressão `2*x-y`.

Neste caso, estamos colocando essa função na variável `f1`, que pode ser usada para a execução da função.

In [86]:
f1

<function __main__.<lambda>>

In [87]:
f1(3,2)

4

Isto seria equivalente à definição abaixo (mas mais conciso): 

In [88]:
def f2(x,y):
    return 2 * x - y

In [89]:
f2(3,2)

4

## Map e filter

Uma situação onde funções lambda são úteis é no uso da funções `map` e `filter`.

`map` recebe uma função e um objeto do qual se podem percorrer os elementos. Ela aplica a função recebida, gerando um elemento para cada elemento original.

In [15]:
x = list(map(lambda x: x//2, [10,20,30,40]))
print(x)

[5, 10, 15, 20]


No código acima, definimos uma função lambda que divide o valor de seu parâmetro por 2. Essa função é aplicada (por `map`) a todos os elementos da lista fornecida. O resultado é coletado em uma nova lista.

Um exemplo similar que usa `range` segue.

In [91]:
list(map(lambda x: 2*x**2, range(10)))

[0, 2, 8, 18, 32, 50, 72, 98, 128, 162]

In [92]:
%timeit list(map(lambda x: 2*x**2, range(10)))

The slowest run took 4.90 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 9.58 µs per loop


In [93]:
%timeit [2*x**2 for x in range(10)]

The slowest run took 5.19 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.99 µs per loop


Se a função fornecida necessita de mais parâmetros, precisamos fornecer uma lista para cada parâmetro.

No codigo abaixo, `map` irá aplicar `f1(1,0), f1(2,1), f1(3,2), ...`

In [94]:
list(map(f1, range(1,11), range(0,10)))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

A função `filter` é similar a `map`, mas a função fornecida como primeiro parâmetro deve retornar um booleano (`True` ou `False`). Essa função é aplicada a cada elemento da coleção fornecida. Cada elemento para o qual a função retornar `True` será inserido no resultado, enquanto os outros serão descartados.

In [95]:
list(filter(lambda x: x % 2 == 0, range(10)))

[0, 2, 4, 6, 8]